In [1]:
# pip install opencv-python
# pip install pytesseract
# pip install imutils
# pip install tensorflow
import cv2 
import pytesseract
import numpy as np

In [2]:
# loading image
img = cv2.imread('Images/cheque_leaf.jpg')

In [3]:
# Rescaling the image (it's recommended if you’re working with images that have a DPI of less than 300 dpi)
img = cv2.resize(img, None, fx=1.2, fy=1.2, interpolation=cv2.INTER_CUBIC)
h, w = img.shape[:2]

In [4]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # transform to grayscale

In [5]:
# To sharpen the image
sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
sharpen = cv2.filter2D(gray, -1, sharpen_kernel)

In [6]:
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]  # perform OTSU threhold
thresh = cv2.rectangle(thresh, (0, 0), (w, h), (0, 0, 0), 2) # draw a rectangle around regions of interest in an image

In [7]:
cv2.imshow("img", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# Dilates an image by using a specific structuring element.
# enrich the charecters(to large)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

#The function erodes the source image using the specified structuring element that determines 
# the shape of a pixel neighborhood over which the minimum is taken
erode = cv2.erode(thresh, kernel, iterations = 1)

In [9]:
cv2.imshow("img", erode)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# save image to file
cv2.imwrite("Images/cheque_wb.jpg", erode)

True

In [10]:
# sudo apt-get install tesseract-ocr-eng 
# Adding custom options -- oem 0-3 psm 0-13
custom_config = r'--oem 1 --psm 6'
pytesseract.image_to_string(erode, config=custom_config)

'0 $.K mens hostel - 324AM-T719AM  §\nWAGR¥7JP, Dhanalakshmi Nagar, Tambaram, Chennal, Tamil Nadu 600059\na Motorcycling - 750 m 2mins  p7\n\n0 Green Fiiness Club( Camp Road) ~ 722AM-7:30AM  §\nNo 2, Sri Ram Nagar 4th Street, Seliyur, post, Tambaram, Chennal, Tamil\nNedu 600073\n\n0 RAWFitness UnissxGym + 7AAM-737AM  §\n7, Vaidyalingam St, behind Annai Violet School, Sudha Avenue,\nChrdapakkam, Chennai, Tamil Nadu 600064\na Motorcycling - 850 m sms #5\n\n0 WAVE FITNESS UNISEX - 7ASAM-T51AM  §\nNo 2,Lenin Complex Bharathi, School St, Camp Road Junction, East\nTambaram, Selalyur, Chennai, Tamil Nadu 600073\n\x0c'

In [17]:
import cv2
import pytesseract
import numpy as np

# Read the original image
img = cv2.imread('Images/cheque_wb.jpg') 
# Display original image
cv2.imshow('Original', img)
cv2.waitKey(0)

# Convert to graycsale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Blur the image for better edge detection
img_blur = cv2.GaussianBlur(img_gray, (3,3), 0) 

# Sobel Edge Detection
sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
# Display Sobel Edge Detection Images
cv2.imshow('Sobel X', sobelx)
cv2.waitKey(0)
cv2.imshow('Sobel Y', sobely)
cv2.waitKey(0)
cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
cv2.waitKey(0)

# Canny Edge Detection
edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection
# Display Canny Edge Detection Image
cv2.imshow('Canny Edge Detection', edges)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [18]:
# save image to file
cv2.imwrite("Images/cheque_edge_wb.jpg", edges)

True

In [19]:
custom_config = r'--oem 1 --psm 6'
pytesseract.image_to_string(edges, config=custom_config)

'SYA) e a at AL\nR= E Ai f a! [aicMCAN= a pO\nANCEBA |\nPINE ONE RACH > Pe ami)\n[Ina 2 Zo! TE\nEe Te IEE D FWE —\nye - YI\nBAND) PARENTER (REEDY a g\n[Createliibiocd Besdeses) pr eeceeloeliete:Slatuek octloao] |\nldellslejsrliHeipialepis clamps oRajoy:Ne\n\x0c'

In [22]:
# import the necessary packages
import numpy as np
import cv2
import pytesseract

def non_max_suppression(boxes, probs=None, overlapThresh=0.3):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)

    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked	return boxes[pick].astype("int")
    return boxes[pick].astype("int")

In [25]:
image = cv2.imread('Images/cheque_bw.jpg')
orig = image.copy()
(H, W) = image.shape[:2]

# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (320, 320)
rW = W / float(newW)
rH = H / float(newH)

# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

# load the pre-trained EAST text detector
net = cv2.dnn.readNet('Images/Model/frozen_east_text_detection.pb')

# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),(123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

# grab the number of rows and columns from the scores volume, then
# initialize our set of bounding box rectangles and corresponding
# confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

# loop over the number of rows
for y in range(0, numRows):
    # extract the scores (probabilities), followed by the geometrical
    # data used to derive potential bounding box coordinates that
    # surround text
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]
    
    for x in range(0, numCols):
        # ignore probability values below 0.75
        if scoresData[x] < 0.75:
            continue
        
        # compute the offset factor as our resulting feature maps will
        # be 4x smaller than the input image
        (offsetX, offsetY) = (x * 4.0, y * 4.0)
        
        # extract the rotation angle for the prediction and then
        # compute the sin and cosine
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)
        
        # use the geometry volume to derive the width and height of
        # the bounding box
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]
        
        # compute both the starting and ending (x, y)-coordinates for
        # the text prediction bounding box
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)
        
        # add the bounding box coordinates and probability score to
        # our respective lists
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
    # scale the bounding box coordinates based on the respective
    # ratios
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)
    # draw the bounding box on the image
    cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
    cv2.imwrite('Images/Crop/crop.png', orig)

In [28]:
# take a copy o the original image
img = cv2.imread('Images/cheque_bw.jpg')
image2 = orig.copy()
for i, (startX, startY, endX, endY) in enumerate(boxes):
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)
    cropped = img[startY:endY, startX:endX]
    cv2.imwrite('Images/Crop/crop_img_{}.jpg'.format(i), cropped)

In [31]:
image = cv2.imread('Images/Crop/crop_img_21.jpg')

custom_config = r'--oem 1 --psm 6'
pytesseract.image_to_string(image, config=custom_config)

'a\n\x0c'

In [32]:
oem = [0,1,2,3]
psm = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

for i in oem:
    for j in psm:
        
        while True:
            try:
                config = r'--oem '+str(i)+' --psm '+str(j)+''
                data = pytesseract.image_to_string(image, config=config)
                print('oem -',str(i),' psm -',str(j),' data-',data)
                break
            except Exception:
                break

oem - 0  psm - 1  data- 
oem - 0  psm - 3  data- 
oem - 0  psm - 4  data- 
oem - 0  psm - 5  data- w
9.“

oem - 0  psm - 6  data- i oﬂgboﬁ

oem - 0  psm - 7  data- i oﬂgboﬁ

oem - 0  psm - 8  data- bdgboﬁ

oem - 0  psm - 9  data- 
oem - 0  psm - 10  data- M

oem - 0  psm - 11  data- ' 0&3‘00/

oem - 0  psm - 12  data- ' 0&3‘00/

oem - 0  psm - 13  data- '0?SDo/—

oem - 1  psm - 1  data- 
oem - 1  psm - 3  data- 
oem - 1  psm - 4  data- 
oem - 1  psm - 5  data- J
&

oem - 1  psm - 6  data- a

oem - 1  psm - 7  data- a

oem - 1  psm - 8  data- Pana

oem - 1  psm - 9  data- Pana

oem - 1  psm - 10  data- Pana

oem - 1  psm - 11  data- a

oem - 1  psm - 12  data- a

oem - 1  psm - 13  data- Pana

oem - 2  psm - 1  data- 
oem - 2  psm - 3  data- 
oem - 2  psm - 4  data- 
oem - 2  psm - 5  data- )
&

oem - 2  psm - 6  data- » o?soo/—

oem - 2  psm - 7  data- » o?soo/—

oem - 2  psm - 8  data- ip?gbo/_

oem - 2  psm - 9  data- 
oem - 2  psm - 10  data- M

oem 

In [38]:
from pytesseract import Output
import pytesseract
import cv2
 
image = cv2.imread("Images/cheque_wb.jpg")

#swap color channel ordering from BGR (OpenCV’s default) to RGB (compatible with Tesseract and pytesseract).
# By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
# we need to convert from BGR to RGB format/mode:
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
custom_config = r'--oem 1 --psm 6'
results = pytesseract.image_to_data(rgb, output_type=Output.DICT,lang='eng',config=custom_config)
boxresults = pytesseract.image_to_boxes(rgb,output_type=Output.DICT,lang='eng',config=custom_config)
print(results['text'])

for i in range(0, len(results["text"])):
    # extract the bounding box coordinates of the text region from the current result
    tmp_tl_x = results["left"][i]
    tmp_tl_y = results["top"][i]
    tmp_br_x = tmp_tl_x + results["width"][i]
    tmp_br_y = tmp_tl_y + results["height"][i] 
    tmp_level = results["level"][i]
    conf = results["conf"][i]
    text = results["text"][i]
    
    if(tmp_level == 5):
        cv2.putText(image, text, (tmp_tl_x, tmp_tl_y - 10), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 255), 1)
        cv2.rectangle(image, (tmp_tl_x, tmp_tl_y), (tmp_br_x, tmp_br_y), (0, 0, 255), 1)
        
for j in range(0,len(boxresults["left"])):
    left = boxresults["left"][j]
    bottom = boxresults["bottom"][j]
    right = boxresults["right"][j]
    top = boxresults["top"][j] 
    cv2.rectangle(image, (left, top), (right, bottom), (255, 0, 0), 1)
       
    
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

['', '', '', '', 'a,', 'WD.', '&6loal2en', '', 'Vay', '%', 'He', '2400', '§', 'Date.', '>', '22', '4', 'mr', '', '2’', 'NCBA', '|', '', 'JUNCTION', 'BRANCH', 'Vs', '0', '07-106', '', 'BAICHANDRY', 'Rep]', 'BanDLUS?', 'fsool—', '', 'i', 'US', 'Dollars', 'Two', 'Tho', '‘', 'ky', 'D', 'FWE', '|', 'BANOLRAT', '.', 'Ree', '', '_PONDRED', 'ALS,', '5', '9', 'A', '', 'BANDI', 'RAVICHANDRA', 'REDDY', '!', ':', '', 'Cheque', 'Number', 'Bank', 'Code', 'Account', 'Number', 'Please', 'DO', 'NOT', 'write', 'in', 'the', 'space', 'below', '1', '', '"000002107', '0E3NEOw3IADLO', 'LOO', '2', '2°']


In [34]:
import cv2
import pytesseract
import numpy as np


# Load image, grayscale, Otsu's threshold
image = cv2.imread('Images/cheque_wb.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Draw bounding boxes
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)

# OCR
data = pytesseract.image_to_string(255 - thresh, lang='eng',config='--oem 1 --psm 6')
print(data)

cv2.imshow('thresh', thresh)
# cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

a, WD. &6loal2en
Vay, % He 2400 § Date. > 22 4 ns
2’ NCBA |
JUNCTION BRANCH VY 0 07-106
BAICWANDRY Rep] BanDLUS? fsool—
i US Dollars Two Tho ‘ ky Dp FWE | BANGIRAT . Ree
_PONDRED ALS, 5 9 A
BANDI RAVICHANDRA REDDY ! :
Cheque Number Bank Code Account Number Please DO NOT write in the space below 1
"0000021207 0E3NEOw3IADLO LOO 2 2



In [27]:
# save image to file
cv2.imwrite("Images/cheque_bw.jpg", thresh)

True